In [1]:
# importing required libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Dropout, ReLU, Flatten, Dense
from tensorflow.keras.models import Model

In [2]:
# importing datasets
train_dir = "../input/emotion-detection-fer/train" # Replace with appropriate path
test_dir = "../input/emotion-detection-fer/test"  

In [3]:
img_size=48
epochs = 60
batch_size = 64

In [4]:
# data augmentation
train_datagen = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
    rescale = 1./255,
    validation_split = 0.2,
    rotation_range = 30
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (img_size,img_size),
    shuffle = True,
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = "categorical",
    subset = "training"
)

validation_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (img_size,img_size),
    shuffle = True,
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = "categorical",
    subset = "validation"
)

test_generator = validation_datagen.flow_from_directory(
    directory = test_dir,
    shuffle = True,
    target_size = (48, 48),
    batch_size = 64,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
def model1(inputLayer):
    x = Conv2D(64, kernel_size=(3, 3))(inputLayer[0])
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(2, 2))(x)
    x = ReLU()(x)
    
    x = Conv2D(64, kernel_size=(3, 3))(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(2, 2))(x)
    x = ReLU()(x)
    
    x = Conv2D(128, kernel_size=(3, 3))(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(2, 2))(x)
    x = ReLU()(x)
    
    x = Conv2D(128, kernel_size=(3, 3))(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(2, 2))(x)
    x = ReLU()(x)
    
    x = Flatten()(x)
    
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(7, activation='softmax')(x)
    
    return x

In [7]:
input_1 = Input(shape=(48, 48, 1), name="input_1")
z_1 = model1([input_1])
model_1 = Model(inputs=[input_1], outputs=[z_1])

model_1.summary()

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
re_lu (ReLU)                 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 21, 21, 64)        256   

In [8]:
def model2(inputLayer):
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputLayer[0])
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (5, 5), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    x = Flatten()(x)
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    x = Dense(7, activation='softmax')(x)
    
    return x

In [9]:
input_2 = Input(shape=(48, 48, 1), name="input_2")
z_2 = model2([input_2])
model_2 = Model(inputs=[input_2], outputs=[z_2])

model_2.summary()

model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 128)       2049

In [10]:
# training model 2
model_2.fit(train_generator,epochs = epochs, validation_data = validation_generator)

Epoch 1/60
359/359 [==============================] - 86s 219ms/step - loss: 9.2972 - accuracy: 0.1887 - val_loss: 8.7952 - val_accuracy: 0.2219
Epoch 2/60
359/359 [==============================] - 35s 98ms/step - loss: 8.2232 - accuracy: 0.2169 - val_loss: 7.5866 - val_accuracy: 0.2390
Epoch 3/60
359/359 [==============================] - 36s 99ms/step - loss: 7.1768 - accuracy: 0.2354 - val_loss: 6.5137 - val_accuracy: 0.2783
Epoch 4/60
359/359 [==============================] - 36s 100ms/step - loss: 6.1540 - accuracy: 0.2552 - val_loss: 5.7025 - val_accuracy: 0.2625
Epoch 5/60
359/359 [==============================] - 35s 98ms/step - loss: 5.2477 - accuracy: 0.2715 - val_loss: 4.7849 - val_accuracy: 0.2865
Epoch 6/60
359/359 [==============================] - 36s 99ms/step - loss: 4.4752 - accuracy: 0.2928 - val_loss: 4.3133 - val_accuracy: 0.2900
Epoch 7/60
359/359 [==============================] - 35s 98ms/step - loss: 3.8524 - accuracy: 0.3129 - val_loss: 3.7980 - val_accurac

In [11]:
# training model 1
model_1.fit(train_generator,epochs = epochs, validation_data = validation_generator)

Epoch 1/60
359/359 [==============================] - 35s 96ms/step - loss: 1.7819 - accuracy: 0.2647 - val_loss: 1.8419 - val_accuracy: 0.1810
Epoch 2/60
359/359 [==============================] - 35s 97ms/step - loss: 1.6474 - accuracy: 0.3441 - val_loss: 1.5890 - val_accuracy: 0.3832
Epoch 3/60
359/359 [==============================] - 34s 94ms/step - loss: 1.5642 - accuracy: 0.3846 - val_loss: 1.5281 - val_accuracy: 0.4017
Epoch 4/60
359/359 [==============================] - 34s 94ms/step - loss: 1.5096 - accuracy: 0.4144 - val_loss: 1.5036 - val_accuracy: 0.4125
Epoch 5/60
359/359 [==============================] - 33s 93ms/step - loss: 1.4516 - accuracy: 0.4369 - val_loss: 1.4216 - val_accuracy: 0.4497
Epoch 6/60
359/359 [==============================] - 34s 94ms/step - loss: 1.4183 - accuracy: 0.4500 - val_loss: 1.4822 - val_accuracy: 0.4358
Epoch 7/60
359/359 [==============================] - 33s 92ms/step - loss: 1.3848 - accuracy: 0.4689 - val_loss: 1.3723 - val_accuracy:

In [12]:
# evaluating model 1
model_1.evaluate(test_generator)

113/113 [==============================] - 30s 267ms/step - loss: 1.0719 - accuracy: 0.5958


[1.0718775987625122, 0.5958484411239624]

In [13]:
# evaluating model 2
model_2.evaluate(test_generator)

113/113 [==============================] - 5s 47ms/step - loss: 1.3315 - accuracy: 0.6410


[1.3314627408981323, 0.6409863233566284]

In [14]:
# saving model_1 in .json file
model_json_1 = model_1.to_json()
with open('my_new_model_1.json', 'w') as file:
    file.write(model_json_1)

# saving weights in .h5 file
model_1.save_weights('my_new_model_1.h5')

In [15]:
# saving model_2 in .json file
model_json_2 = model_2.to_json()
with open('my_new_model_2.json', 'w') as file:
    file.write(model_json_2)

# saving weights in .h5 file
model_2.save_weights('my_new_model_2.h5')